In [1]:
import os
import sys
import random
import json
import collections
import re
from itertools import combinations

import pandas as pd
import numpy as np
import scipy
import statsmodels
from tqdm import trange, tqdm_notebook as tqdm

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report
from lightgbm import LGBMClassifier

from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline


pd.options.display.max_columns = 999

sns.set()
plt.rcParams["figure.figsize"] = (10,7)

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from sigmod_src.utils import read_json, pprint_json, get_known_brands
from sigmod_src.data.make_dataset import make_specs_dataset, preprocess_specs_dataset, join_labels_specs
from sigmod_src.features.build_features import make_features

[nltk_data] Downloading package punkt to /home/boris/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
LG_LABELS_PATH = '../data/raw/sigmod_large_labelled_dataset.csv'

In [5]:
labels_df = pd.read_csv(LG_LABELS_PATH)
labels_df.shape

(297651, 3)

In [6]:
labels_df.head()

,left_spec_id,right_spec_id,label
0,www.ebay.com//53278,www.garricks.com.au//31,1
1,www.ebay.com//24817,www.ebay.com//58782,0
2,www.ebay.com//43019,www.ebay.com//58782,0
3,www.ebay.com//42055,www.ebay.com//54403,0
4,buy.net//6145,www.ebay.com//44280,0


In [7]:
SPECS_PATH = '../data/raw/2013_camera_specs/'

### Get Brands for all products

Getting all titles and brands form specs

In [8]:
site_folders = os.listdir(SPECS_PATH)

In [9]:
specs_dataset_src = make_specs_dataset(SPECS_PATH)

In [10]:
specs_dataset_src.head()

,spec_id,page_title,brand,all_text
0,www.ebay.com//57656,Canon PowerShot ELPH 110 HS 16 1 MP Digital Ca...,canon,Canon PowerShot ELPH 110 HS 16 1 MP Digital Ca...
1,www.ebay.com//60583,Canon Rebel 2000 35 mm Camera Great Condition ...,canon,Canon Rebel 2000 35 mm Camera Great Condition ...
2,www.ebay.com//60440,Canon EOS Rebel T3i Digital SLR Camera 18 55mm...,None,Canon EOS Rebel T3i Digital SLR Camera 18 55mm...
3,www.ebay.com//24139,"GE C1033 10 1 MP Digital Camera 3X Zoom 2 4"" L...",ge,"GE C1033 10 1 MP Digital Camera 3X Zoom 2 4"" L..."
4,www.ebay.com//54903,Vivitar Clip Shot Digital Camera 1 1 MP | eBay,vivitar,Vivitar Clip Shot Digital Camera 1 1 MP | eBay...


In [11]:
specs_dataset_src.shape

(29787, 4)

In [12]:
specs_dataset_src[specs_dataset_src.brand.isnull()].shape

(14048, 4)

In [13]:
specs_dataset_src[specs_dataset_src.brand.isnull()].head()

,spec_id,page_title,brand,all_text
2,www.ebay.com//60440,Canon EOS Rebel T3i Digital SLR Camera 18 55mm...,None,Canon EOS Rebel T3i Digital SLR Camera 18 55mm...
32,www.ebay.com//46156,Sony Cyber Shot 63X Super Zoom DSC H400 HD Dig...,None,Sony Cyber Shot 63X Super Zoom DSC H400 HD Dig...
82,www.ebay.com//55668,Sony Cyber Shot DSC W710 16 1 MP Digital Camer...,None,Sony Cyber Shot DSC W710 16 1 MP Digital Camer...
110,www.ebay.com//52947,Sony NEX 5 14 2MP Digital Camera Body w 18 55 ...,None,Sony NEX 5 14 2MP Digital Camera Body w 18 55 ...
170,www.ebay.com//47024,On Stock Original Sjcam SJ4000 WiFi HD1080P Sp...,None,On Stock Original Sjcam SJ4000 WiFi HD1080P Sp...


In [14]:
specs_dataset_src.brand.value_counts()

canon       3914
nikon       3365
sony        2111
olympus      948
fujifilm     935
            ... 
computer       1
dnt            1
venturer       1
tandberg       1
genius         1
Name: brand, Length: 250, dtype: int64

## Preprocess text fields:
* lowercase
* fix known common typos
* remove nltk stopwords + some hand-picked stop words
* remove digit sequences of length 10 and more
* only ascii symbols
* only `string.printable` symbols
* only alphanumeric symbols
* max 500 words
* remove words that appear only once
* extract brand from title if its not present
* drop rows where page title is empty or null

In [15]:
brand_blacklist = ['shoot', 'unbranded/generic', 'as', 
            'eos', 'action', 'new', 'class', 'neopine', 'sharp',
            'telesin']
brand_cutoff = 10 # Remove brands appear in < brand_cutoff specs
cutoff = 1
max_words = 500

In [16]:
known_brands = get_known_brands(specs_dataset_src, brand_cutoff, brand_blacklist)
len(known_brands)

37

In [17]:
np.array(known_brands)

array(['canon', 'nikon', 'sony', 'olympus', 'fujifilm', 'panasonic',
       'samsung', 'kodak', 'pentax', 'vivitar', 'casio', 'leica',
       'polaroid', 'ge', 'hp', 'ricoh', 'carbose', 'gopro', 'konica',
       'svp', 'sanyo', 'fuji', 'vizio', 'digital', 'minolta', 'coleman',
       'fvanor', 'sakar', 'bell', 'lg', 'sigma', 'toshiba', 'philips',
       'lytro', 'sealife', 'hasselblad', 'cobra'], dtype='<U10')

In [18]:
specs_dataset_src[['brand', 'all_text']].head().values

array([['canon',
        'Canon PowerShot ELPH 110 HS 16 1 MP Digital Camera Black | eBay\tCanon\tManufacturer refurbished: An item that has been professionally restored to working order by a manufacturer or\nmanufacturer-approved vendor. This means the product has been inspected, cleaned, and repaired to meet manufacturer specifications and is in excellent condition. This item may or may not be in the original packaging. See the sellerâ\x80\x99s listing for full details.\nSee all condition definitions- opens in a new window or tab\n... Read moreabout the condition\t16.1 MP\tELPH 110 HS\tE262121\tPoint & Shoot'],
       ['canon',
        'Canon Rebel 2000 35 mm Camera Great Condition with Case and Instruction Booklet | eBay\tCanon\tCase or Bag\tTaiwan\tNo\tna\trebel 2000\t21-69008757'],
       [None,
        'Canon EOS Rebel T3i Digital SLR Camera 18 55mm Is 75 300mm III Lens Kit USA 609728170592 | eBay\tYes'],
       ['ge',
        'GE C1033 10 1 MP Digital Camera 3X Zoom 2 4" LCD Bla

In [19]:
specs_dataset = preprocess_specs_dataset(specs_dataset_src,
                                         max_words=max_words,
                                         cutoff=cutoff,
                                         known_brands=known_brands,
                                        brand_blacklist=brand_blacklist,
                                        brand_cutoff=5)

Conflict. Found: sigma , brand field: nikon  Will use brand field
Conflict. Found: sony , brand field: olympus  Will use brand field
Conflict. Found: sony , brand field: kodak  Will use brand field
Conflict. Found: sony , brand field: olympus  Will use brand field
Conflict. Found: samsung , brand field: vivitar  Will use brand field
Conflict. Found: olympus , brand field: panasonic  Will use brand field
Conflict. Found: canon , brand field: fujifilm  Will use brand field
Conflict. Found: sigma , brand field: nikon  Will use brand field
Conflict. Found: sony , brand field: olympus  Will use brand field
Conflict. Found: gopro , brand field: ion  Will use brand field
Conflict. Found: konica , brand field: minolta  Will use brand field
Conflict. Found: canon , brand field: casio  Will use brand field
Conflict. Found: kodak , brand field: vivitar  Will use brand field
Conflict. Found: sigma , brand field: nikon  Will use brand field
Conflict. Found: canon , brand field: olympus  Will use br

In [20]:
specs_dataset.head()

,spec_id,page_title,brand,all_text,page_title_stem,all_text_stem,site
0,www.ebay.com//57656,canon powershot elph 110 hs 16 1 mp,canon,canon powershot 110 hs 16 1 mp canon refurbish...,canon powershot elph 110 hs 16 1 mp,canon powershot 110 hs 16 1 mp canon refurbish...,www.ebay.com
1,www.ebay.com//60583,canon rebel 2000 35 mm great case instruction ...,canon,canon rebel 2000 35 mm great case instruction ...,canon rebel 2000 35 mm great case instruct boo...,canon rebel 2000 35 mm great case instruct boo...,www.ebay.com
2,www.ebay.com//60440,canon eos rebel t3i 18 55mm 75 300mm iii lens ...,canon,canon eos rebel t3i 18 55mm 75 300mm iii lens ...,canon eo rebel t3i 18 55mm 75 300mm iii len ki...,canon eo rebel t3i 18 55mm 75 300mm iii len ki...,www.ebay.com
3,www.ebay.com//24139,ge c1033 10 1 mp 3x zoom 2 4 lcd,ge,ge c1033 10 1 mp 3x zoom 2 4 ge applicable fou...,ge c1033 10 1 mp 3x zoom 2 4 lcd,ge c1033 10 1 mp 3x zoom 2 4 ge applic found r...,www.ebay.com
4,www.ebay.com//54903,vivitar clip shot 1 1 mp,vivitar,vivitar clip shot 1 1 mp vivitar applicable fo...,vivitar clip shot 1 1 mp,vivitar clip shot 1 1 mp vivitar applic found ...,www.ebay.com


In [21]:
specs_dataset[['brand', 'all_text']].sample(5).values

array([['canon',
        'canon eos 700d 18 0 mp body canon may signs cosmetic wear fully operational functions intended may floor model return full description imperfections mp 700d ds126431'],
       ['canon',
        'canon powershot sd780 ixus 100 12 1 mp canon applicable found retail unless handmade packaged unprinted box plastic bag full mp sd780 ixus 100 3590b001 3x point shoot'],
       ['canon',
        'canon rebel t2i eos 550d 18 0 mp canon case bag memory card may signs cosmetic wear fully operational functions intended may floor model return full description imperfections mp rebel t2i'],
       [None,
        '3g alarm wireless video alarm gsm alarm wireless video wireless alarm get latest 300 western union xiamen 50000 per month'],
       ['panasonic',
        'panasonic lumix point shoot india offers full specifications color panasonic usb usb hdmi yes hdmi mini connector bluetooth gps dimensions inches weight 184 gms display type touchscreen screen size 3 inches image d

## Join specs with labels

In [22]:
labels_df.shape

(297651, 3)

In [23]:
labels_specs_df = join_labels_specs(labels_df, specs_dataset)
labels_specs_df.shape

(297651, 15)

In [24]:
labels_specs_df.head()

,left_spec_id,right_spec_id,label,left_page_title,left_brand,left_all_text,left_page_title_stem,left_all_text_stem,left_site,right_page_title,right_brand,right_all_text,right_page_title_stem,right_all_text_stem,right_site
0,www.ebay.com//53278,www.garricks.com.au//31,1,nikon d3200 24 2 mp kit w af dx 18 55mm lens,nikon,d3200 24 2 mp kit w af dx 18 55mm lens may sig...,nikon d3200 24 2 mp kit w af dx 18 55mm len,d3200 24 2 mp kit w af dx 18 55mm len may sign...,www.ebay.com,nikon d3200 vr lens,None,d3200 vr lens inbuilt dx cmos,nikon d3200 vr len,d3200 vr len inbuilt dx cmos,www.garricks.com.au
1,www.ebay.com//24817,www.ebay.com//58782,0,fujifilm finepix s1 weather resistant 16 mp 16...,fujifilm,fujifilm finepix s1 weather resistant 16 mp 16...,fujifilm finepix s1 weather resist 16 mp 16408967,fujifilm finepix s1 weather resist 16 mp 16408...,www.ebay.com,nikon d80 10 2 mp kit w af dx ed g 18 70mm,nikon,d80 10 2 mp kit w af dx ed g 18 70mm may signs...,nikon d80 10 2 mp kit w af dx ed g 18 70mm,d80 10 2 mp kit w af dx ed g 18 70mm may sign ...,www.ebay.com
2,www.ebay.com//43019,www.ebay.com//58782,0,nikon d3200 24 2 mp body 30717,nikon,d3200 24 2 mp body 30717 may signs cosmetic we...,nikon d3200 24 2 mp bodi 30717,d3200 24 2 mp bodi 30717 may sign cosmet wear ...,www.ebay.com,nikon d80 10 2 mp kit w af dx ed g 18 70mm,nikon,d80 10 2 mp kit w af dx ed g 18 70mm may signs...,nikon d80 10 2 mp kit w af dx ed g 18 70mm,d80 10 2 mp kit w af dx ed g 18 70mm may sign ...,www.ebay.com
3,www.ebay.com//42055,www.ebay.com//54403,0,nikon d80 body acessories 2 battery packs,nikon,d80 body acessories 2 battery packs case bag e...,nikon d80 bodi acessori 2 batteri pack,d80 bodi acessori 2 batteri pack case bag extr...,www.ebay.com,nikon d3200 24 2mp body 301,nikon,d3200 24 2mp body may signs cosmetic wear full...,nikon d3200 24 2mp bodi 301,d3200 24 2mp bodi may sign cosmet wear fulli o...,www.ebay.com
4,buy.net//6145,www.ebay.com//44280,0,canon powershot sx600 hs 16 megapixel compact ...,canon,canon powershot sx600 hs 16 megapixel compact ...,canon powershot sx600 hs 16 megapixel compact ...,canon powershot sx600 hs 16 megapixel compact ...,buy.net,nikon d3200 24 2mp body 3 year warranty,nikon,d3200 24 2mp body 3 year warranty please inclu...,nikon d3200 24 2mp bodi 3 year warranti,d3200 24 2mp bodi 3 year warranti pleas includ...,www.ebay.com


## Check whether dublicates have the same brand

In [25]:
non_dup_df = labels_specs_df[labels_specs_df.label==0]
dup_df = labels_specs_df[labels_specs_df.label==1]

dup_missing_brand = dup_df[dup_df.left_brand.isnull() | dup_df.right_brand.isnull()]
dup_same_brand = dup_df[dup_df.left_brand == dup_df.right_brand]
dup_diff_brand = dup_df[dup_df.left_brand != dup_df.right_brand]

non_dup_missing_brand = non_dup_df[non_dup_df.left_brand.isnull() | non_dup_df.right_brand.isnull()]
non_dup_same_brand = non_dup_df[non_dup_df.left_brand == non_dup_df.right_brand]
non_dup_diff_brand = non_dup_df[non_dup_df.left_brand != non_dup_df.right_brand]

In [26]:
print('Total duplicates', dup_df.shape[0])
print('Total non-duplicates', non_dup_df.shape[0])

print('Duplicates with missing brand', dup_missing_brand.shape[0])
print('Duplicates with same brand', dup_same_brand.shape[0], round(dup_same_brand.shape[0]/dup_df.shape[0], 2))
print('Duplicates with diff brands', dup_diff_brand.shape[0])

print('Non-duplicates with missing brand', non_dup_missing_brand.shape[0])
print('Non-duplicates with same brand', non_dup_same_brand.shape[0], round(non_dup_same_brand.shape[0]/non_dup_df.shape[0], 2))
print('Non-duplicates with diff brands', non_dup_diff_brand.shape[0])

Total duplicates 44039
Total non-duplicates 253612
Duplicates with missing brand 8198
Duplicates with same brand 35841 0.81
Duplicates with diff brands 8198
Non-duplicates with missing brand 55234
Non-duplicates with same brand 80390 0.32
Non-duplicates with diff brands 173222


Good news! 99% duplicates have same brand! And most non-duplicates have different brands! And most duplicates don't have missing brands!

### Get model for all products

In [27]:
# from collections import defaultdict
# models = defaultdict(list)
# z = 0
# for site in site_folders:
#     specs = os.path.join(SPECS_PATH, site)
#     for i, spec in enumerate(os.listdir(specs)):
#         spec_path = os.path.join(specs, spec)
#         js = read_json(spec_path)
#         model = js.get('model')
#         if isinstance(model, list):
#             model = ' '.join(model)
#         if model:
#             model = re.sub('[^a-z0-9] ', '', model.lower())
#             models[model].append(z)
#         z +=1

# Make classifier features

In [28]:
specs_dataset.brand.value_counts()

canon       5589
sony        3399
nikon       3363
fujifilm    1512
olympus     1456
            ... 
toi            5
ion            5
tamron         5
mustek         5
godspeed       5
Name: brand, Length: 66, dtype: int64

In [29]:
specs_dataset['brand'] = specs_dataset['brand'].fillna('missing')
spec_features_df, vectorizers, label_encoders = make_features(specs_dataset)
spec_features_df.index = spec_features_df.spec_id
spec_features_df.shape

/home/boris/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


(29771, 503)

In [30]:
label_encoders['brand'].classes_

array(['aiptek', 'argus', 'bell', 'benq', 'canon', 'carbose', 'casio',
       'cobra', 'coleman', 'disney', 'dxg', 'easypix', 'emerson', 'epson',
       'fotopix', 'fuji', 'fujifilm', 'fvanor', 'garmin', 'ge',
       'godspeed', 'gopro', 'hasselblad', 'hp', 'insignia', 'intova',
       'ion', 'jvc', 'kodak', 'konica', 'leica', 'lg', 'lowrance',
       'lytro', 'magnavox', 'minolta', 'minox', 'missing', 'mustek',
       'naxa', 'nikon', 'oem', 'olympus', 'oregon', 'panasonic', 'pentax',
       'philips', 'polaroid', 'rca', 'ricoh', 'sakar', 'samsung', 'sanyo',
       'sealife', 'sigma', 'sj4000', 'sony', 'superheadz', 'svp',
       'tamron', 'toi', 'toshiba', 'vistaquest', 'vivitar', 'vizio',
       'vtech', 'yourdeal'], dtype=object)

In [31]:
spec_features_df.head()

,page_title_stem__10,page_title_stem__10 cctv,page_title_stem__10 mp,page_title_stem__1080p,page_title_stem__1080p hd,page_title_stem__1080p ip,page_title_stem__10x,page_title_stem__12,page_title_stem__12 mp,page_title_stem__12mp,page_title_stem__14,page_title_stem__14 mp,page_title_stem__14mp,page_title_stem__16,page_title_stem__16 mp,page_title_stem__16gb,page_title_stem__16mp,page_title_stem__18,page_title_stem__18 55,page_title_stem__18 55mm,page_title_stem__18 mp,page_title_stem__1mp,page_title_stem__20,page_title_stem__20 mp,page_title_stem__2014,page_title_stem__2014 spec,page_title_stem__20x,page_title_stem__24,page_title_stem__24 mp,page_title_stem__2mp,page_title_stem__300,page_title_stem__32gb,page_title_stem__3mp,page_title_stem__3x,page_title_stem__4x,page_title_stem__50mm,page_title_stem__55,page_title_stem__55mm,page_title_stem__55mm len,page_title_stem__5d,page_title_stem__5d mark,page_title_stem__5mp,page_title_stem__5x,page_title_stem__600d,page_title_stem__60d,page_title_stem__700tvl,page_title_stem__70d,page_title_stem__720p,page_title_stem__75,page_title_stem__7d,page_title_stem__accessori,page_title_stem__action,page_title_stem__af,page_title_stem__af dx,page_title_stem__ahd,page_title_stem__ahmedabad,page_title_stem__ahmedabad surat,page_title_stem__alarm,page_title_stem__alpha,page_title_stem__angl,page_title_stem__australia,page_title_stem__bag,page_title_stem__bag bag,page_title_stem__bag case,page_title_stem__bag waterproof,page_title_stem__bangalor,page_title_stem__bangalor hyderabad,page_title_stem__batteri,page_title_stem__best,page_title_stem__best india,page_title_stem__bodi,page_title_stem__box,page_title_stem__bullet,page_title_stem__bullet hikvis,page_title_stem__bullet ip,page_title_stem__bundl,page_title_stem__camcord,page_title_stem__camera,page_title_stem__camera australia,page_title_stem__camera canon,page_title_stem__camera point,page_title_stem__cammarkt,page_title_stem__canon,page_title_stem__canon eo,page_title_stem__canon europ,page_title_stem__canon ixus,page_title_stem__canon powershot,page_title_stem__car,page_title_stem__card,page_title_stem__card slot,page_title_stem__carri,page_title_stem__carri case,page_title_stem__case,page_title_stem__case bag,page_title_stem__case canon,page_title_stem__case case,page_title_stem__case eva,page_title_stem__case gopro,page_title_stem__case hard,page_title_stem__case waterproof,page_title_stem__casio,page_title_stem__casio exilim,page_title_stem__ccd,page_title_stem__cctv,page_title_stem__cctv camera,page_title_stem__cctv cctv,page_title_stem__cctv hikvis,page_title_stem__cctv ip,page_title_stem__cctv secur,page_title_stem__charger,page_title_stem__cheap,page_title_stem__chennai,page_title_stem__chennai kolkata,page_title_stem__chennai mumbai,page_title_stem__china,page_title_stem__cmos,page_title_stem__color,page_title_stem__color ccd,page_title_stem__compact,page_title_stem__compact camera,page_title_stem__comparison,page_title_stem__comparison camera,page_title_stem__comparison nikon,page_title_stem__connect,page_title_stem__coolpix,page_title_stem__cover,page_title_stem__custom,page_title_stem__cyber,page_title_stem__cyber shot,page_title_stem__cybershot,page_title_stem__cybershot dsc,page_title_stem__d3100,page_title_stem__d3200,page_title_stem__d5100,page_title_stem__d5200,page_title_stem__d7000,page_title_stem__dahua,page_title_stem__dahua ip,page_title_stem__dahua ptz,page_title_stem__delhi,page_title_stem__delhi chennai,page_title_stem__delhi mumbai,page_title_stem__design,page_title_stem__detect,page_title_stem__dive,page_title_stem__dmc,page_title_stem__dome,page_title_stem__dome cctv,page_title_stem__dome dahua,page_title_stem__dome hikvis,page_title_stem__dome ip,page_title_stem__dome ir,page_title_stem__dsc,page_title_stem__dual,page_title_stem__dv,page_title_stem__dvr,page_title_stem__dx,page_title_stem__easyshar,page_title_stem__ed,page_title_stem__ef,page_title_stem__ef 18,page_title_stem__elph,page_title_stem__eo,page_title

In [32]:
labels_features = join_labels_specs(labels_df, spec_features_df)

In [33]:
labels_features.shape

(297651, 1007)

In [34]:
labels_df.shape

(297651, 3)

In [35]:
labels_features.head()

,left_spec_id,right_spec_id,label,left_page_title_stem__10,left_page_title_stem__10 cctv,left_page_title_stem__10 mp,left_page_title_stem__1080p,left_page_title_stem__1080p hd,left_page_title_stem__1080p ip,left_page_title_stem__10x,left_page_title_stem__12,left_page_title_stem__12 mp,left_page_title_stem__12mp,left_page_title_stem__14,left_page_title_stem__14 mp,left_page_title_stem__14mp,left_page_title_stem__16,left_page_title_stem__16 mp,left_page_title_stem__16gb,left_page_title_stem__16mp,left_page_title_stem__18,left_page_title_stem__18 55,left_page_title_stem__18 55mm,left_page_title_stem__18 mp,left_page_title_stem__1mp,left_page_title_stem__20,left_page_title_stem__20 mp,left_page_title_stem__2014,left_page_title_stem__2014 spec,left_page_title_stem__20x,left_page_title_stem__24,left_page_title_stem__24 mp,left_page_title_stem__2mp,left_page_title_stem__300,left_page_title_stem__32gb,left_page_title_stem__3mp,left_page_title_stem__3x,left_page_title_stem__4x,left_page_title_stem__50mm,left_page_title_stem__55,left_page_title_stem__55mm,left_page_title_stem__55mm len,left_page_title_stem__5d,left_page_title_stem__5d mark,left_page_title_stem__5mp,left_page_title_stem__5x,left_page_title_stem__600d,left_page_title_stem__60d,left_page_title_stem__700tvl,left_page_title_stem__70d,left_page_title_stem__720p,left_page_title_stem__75,left_page_title_stem__7d,left_page_title_stem__accessori,left_page_title_stem__action,left_page_title_stem__af,left_page_title_stem__af dx,left_page_title_stem__ahd,left_page_title_stem__ahmedabad,left_page_title_stem__ahmedabad surat,left_page_title_stem__alarm,left_page_title_stem__alpha,left_page_title_stem__angl,left_page_title_stem__australia,left_page_title_stem__bag,left_page_title_stem__bag bag,left_page_title_stem__bag case,left_page_title_stem__bag waterproof,left_page_title_stem__bangalor,left_page_title_stem__bangalor hyderabad,left_page_title_stem__batteri,left_page_title_stem__best,left_page_title_stem__best india,left_page_title_stem__bodi,left_page_title_stem__box,left_page_title_stem__bullet,left_page_title_stem__bullet hikvis,left_page_title_stem__bullet ip,left_page_title_stem__bundl,left_page_title_stem__camcord,left_page_title_stem__camera,left_page_title_stem__camera australia,left_page_title_stem__camera canon,left_page_title_stem__camera point,left_page_title_stem__cammarkt,left_page_title_stem__canon,left_page_title_stem__canon eo,left_page_title_stem__canon europ,left_page_title_stem__canon ixus,left_page_title_stem__canon powershot,left_page_title_stem__car,left_page_title_stem__card,left_page_title_stem__card slot,left_page_title_stem__carri,left_page_title_stem__carri case,left_page_title_stem__case,left_page_title_stem__case bag,left_page_title_stem__case canon,left_page_title_stem__case case,left_page_title_stem__case eva,left_page_title_stem__case gopro,left_page_title_stem__case hard,left_page_title_stem__case waterproof,left_page_title_stem__casio,left_page_title_stem__casio exilim,left_page_title_stem__ccd,left_page_title_stem__cctv,left_page_title_stem__cctv camera,left_page_title_stem__cctv cctv,left_page_title_stem__cctv hikvis,left_page_title_stem__cctv ip,left_page_title_stem__cctv secur,left_page_title_stem__charger,left_page_title_stem__cheap,left_page_title_stem__chennai,left_page_title_stem__chennai kolkata,left_page_title_stem__chennai mumbai,left_page_title_stem__china,left_page_title_stem__cmos,left_page_title_stem__color,left_page_title_stem__color ccd,left_page_title_stem__compact,left_page_title_stem__compact camera,left_page_title_stem__comparison,left_page_title_stem__comparison camera,left_page_title_stem__comparison nikon,left_page_title_stem__connect,left_page_title_stem__coolpix,left_page_title_stem__cover,left_page_title_stem__custom,left_page_title_stem__cyber,left_page_title_stem__cyber shot,left_page_title_stem__cybershot,left_page_title_stem__cybershot dsc,left_page_title_stem__d3100,left_page_title_stem__d3200,left_page_title_stem__d

## Clear memory

In [36]:
del non_dup_df
del dup_df
del dup_missing_brand
del dup_same_brand
del dup_diff_brand
del non_dup_missing_brand
del non_dup_same_brand
del non_dup_diff_brand
del labels_specs_df
del labels_df

In [37]:
# print memory usage

# These are the usual ipython objects, including this one you are creating
ipython_vars = ['In', 'Out', 'exit', 'quit', 'get_ipython', 'ipython_vars']

# Get a sorted list of the objects and their sizes
sorted([(x, sys.getsizeof(globals().get(x))) for x in dir() if not x.startswith('_') and x not in sys.modules and x not in ipython_vars], key=lambda x: x[1], reverse=True)

[('labels_features', 2441248448),
 ('spec_features_df', 124188446),
 ('specs_dataset', 41396842),
 ('specs_dataset_src', 39516753),
 ('LGBMClassifier', 1056),
 ('combinations', 400),
 ('known_brands', 360),
 ('site_folders', 264),
 ('label_encoders', 240),
 ('vectorizers', 240),
 ('brand_blacklist', 144),
 ('classification_report', 136),
 ('f1_score', 136),
 ('get_known_brands', 136),
 ('join_labels_specs', 136),
 ('make_features', 136),
 ('make_specs_dataset', 136),
 ('pprint_json', 136),
 ('preprocess_specs_dataset', 136),
 ('read_json', 136),
 ('train_test_split', 136),
 ('trange', 136),
 ('LG_LABELS_PATH', 94),
 ('np', 80),
 ('pd', 80),
 ('plt', 80),
 ('sns', 80),
 ('SPECS_PATH', 79),
 ('brand_cutoff', 28),
 ('cutoff', 28),
 ('max_words', 28)]

# Train/test split

In [38]:
Y = labels_features.label
labels_features.drop(['label', 'left_spec_id', 'right_spec_id'], axis=1, inplace=True)
X = labels_features

In [39]:
train_X, test_X, train_Y, test_Y = train_test_split(X, Y, stratify=Y)
train_Y.shape, test_Y.shape

((223238,), (74413,))

In [40]:
train_Y.value_counts()

0    190209
1     33029
Name: label, dtype: int64

In [41]:
test_Y.value_counts()

0    63403
1    11010
Name: label, dtype: int64

In [42]:
sample_pos_weight = Y.value_counts()[0] / Y.value_counts()[1]
sample_pos_weight

5.75880469583778

In [43]:
clf = LGBMClassifier(sample_pos_weight=sample_pos_weight, n_jobs=-1)

In [44]:
clf.fit(train_X, train_Y)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               learning_rate=0.1, max_depth=-1, min_child_samples=20,
               min_child_weight=0.001, min_split_gain=0.0, n_estimators=100,
               n_jobs=-1, num_leaves=31, objective=None, random_state=None,
               reg_alpha=0.0, reg_lambda=0.0,
               sample_pos_weight=5.75880469583778, silent=True, subsample=1.0,
               subsample_for_bin=200000, subsample_freq=1)

In [45]:
pred_test_Y = clf.predict(test_X)

In [46]:
print('Test F1', f1_score(test_Y, pred_test_Y))
print(classification_report(test_Y, pred_test_Y))

Test F1 0.950281954887218
              precision    recall  f1-score   support

           0       0.99      1.00      0.99     63403
           1       0.98      0.92      0.95     11010

    accuracy                           0.99     74413
   macro avg       0.99      0.96      0.97     74413
weighted avg       0.99      0.99      0.99     74413



# Submit

In [47]:
specs_dataset['spec_idx'] = range(len(specs_dataset))
specs_dataset.index = specs_dataset.spec_idx

/home/boris/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [48]:
brand_groups = specs_dataset.groupby('brand')['spec_idx'].agg(list).to_dict()
brand_groups['canon'][:10]

[0, 1, 2, 9, 11, 12, 13, 15, 18, 22]

In [49]:
OUT_FILE_PATH = '../data/submit/submit.csv'

if os.path.exists(OUT_FILE_PATH):
    os.remove(OUT_FILE_PATH)

batch_size = 5000

In [50]:
def write_chunk_df(chunk_df):
    out_df = chunk_df[chunk_df.label == 1][['left_spec_id', 'right_spec_id']]
    if not out_df.empty:
        if os.path.exists(OUT_FILE_PATH):
            out_df.to_csv(OUT_FILE_PATH, mode='a', header=False, index=False)
        else:
            out_df.to_csv(OUT_FILE_PATH, index=False)

In [51]:
feature_columns = list(spec_features_df.columns)
feature_columns.remove('spec_id')

In [52]:
row_cols = ['left_'+col for col in feature_columns]+['right_'+col for col in feature_columns]
assert len(row_cols) == X.shape[1]

In [53]:
spec_features_arr = spec_features_df.drop('spec_id', axis=1).values
spec_ids_arr = spec_features_df.spec_id.values

In [54]:
for brand, group_specs in tqdm(brand_groups.items()):
    print(brand)
    features_row_buffer = []
    ids_row_buffer = []
    brand_combs = np.array(list(combinations(group_specs, 2)))
    for i in tqdm(range(0, len(brand_combs), batch_size)):
        batch_left_spec_idxs = brand_combs[i:i+batch_size][:, 0]
        batch_right_spec_idxs = brand_combs[i:i+batch_size][:, 1]
        
        batch_left_spec_ids = spec_ids_arr[batch_left_spec_idxs]
        batch_right_spec_ids = spec_ids_arr[batch_right_spec_idxs]
        
        left_features = spec_features_arr[batch_left_spec_idxs]
        right_features = spec_features_arr[batch_right_spec_idxs]
        
        chunk_x = np.hstack([left_features, right_features])
        chunk_df = pd.DataFrame(chunk_x, columns=row_cols)
        chunk_df['label'] = clf.predict(chunk_x)
        chunk_df['left_spec_id'] = batch_left_spec_ids
        chunk_df['right_spec_id'] = batch_right_spec_ids
        write_chunk_df(chunk_df)

HBox(children=(IntProgress(value=0, max=67), HTML(value='')))

aiptek


HBox(children=(IntProgress(value=0, max=1), HTML(value='')))

argus


HBox(children=(IntProgress(value=0, max=1), HTML(value='')))

...

HBox(children=(IntProgress(value=0, max=1), HTML(value='')))

vtech


HBox(children=(IntProgress(value=0, max=1), HTML(value='')))

yourdeal


HBox(children=(IntProgress(value=0, max=1), HTML(value='')))

In [58]:
submit_df = pd.read_csv('../data/submit/submit.csv')
submit_df.shape

(1594264, 2)

In [59]:
specs_dataset.index = specs_dataset.spec_id

In [64]:
row = submit_df.sample(1)

print('Left\n', specs_dataset.loc[row.left_spec_id].all_text.values[0])
print('')
print('Right\n', specs_dataset.loc[row.right_spec_id].all_text.values[0])

Left
 coolpix 4300 4 0 mp may signs cosmetic wear fully operational functions intended may floor model return full description imperfections mp 4300

Right
 coolpix s6500 16 0 mp latest model memory card may signs cosmetic wear fully operational functions intended may floor model return full description imperfections states mp s6500 26372 12x 3 point shoot
